# Libraries

In [ ]:


# Core libraries
import os
import io
import re
import json
import ast
import time
import zipfile
import warnings
import math
import pickle
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Data handling
import numpy as np
import pandas as pd

# Excel handling
import openpyxl
import pyxlsb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tools
from scipy.stats import entropy, norm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Binning and feature engineering
import optbinning
from optbinning import BinningProcess, ContinuousOptimalBinning, OptimalBinning

# Machine learning - Scikit-learn
from sklearn.model_selection import (
    train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
)

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,
)

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics, preprocessing
from sklearn.datasets import make_classification
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    make_scorer, balanced_accuracy_score, roc_curve
)

from catboost import CatBoostClassifier

# XGBoost & LightGBM
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Streamlit (for UI/app development)
import streamlit as st
from streamlit.runtime.scriptrunner import RerunException, get_script_run_ctx


# Folder Path

In [ ]:
path_main=rf"D:\Pranto\Working\Collection Model\Collection New Model\3 to 3 Plus PV\Data\V1\3 to 3 Plus PV\Data"

# Main data file name
file_name_main = "3 TO 3 PLUS JAN24 TO MAR25_PV_MODEL_Overall Data_apply_binning_Data"

# import

In [ ]:
try:
    df=pd.read_csv(rf"{path_main}\{file_name_main}.csv",encoding='latin-1')

except:
    df=pd.read_excel(rf"{path_main}\{file_name_main}.xlsx")

# data shape
print(f"Data shape: {df.shape}")
df.head(3)

In [ ]:
df.columns.tolist()

# Input

In [ ]:
columns_name_selcetion = '_WOE'
columns_last_name = [col for col in df.columns if columns_name_selcetion in col]
columns_last_name

In [ ]:
df['MONTH'].unique()

In [ ]:
# Model used variables
# final_variables_list=columns_last_name #Variable with last name _WOE
final_variables_list=[  #Variable without last name _WOE
'PDC_FLAG_WOE',
 'ASSET_CATEGORY_WOE',
 'BRANCH_WOE',
 'DSA_DLR_WOE',
 'Restructuring_Flag_WOE',
 'PRODUCT_New_WOE',
 'APPLICATION_USAGE_WOE',
 'CUSTOMER_SEGMENT_WOE',
 'SOURCING_WOE',
 'Seasonality_WOE',
 'IRR_CUSTOMER_WOE',
 'ASSET_COST_WOE',
 'MOB_WOE',
 'max_odbkt_WOE',
 'Ever_90_WOE',
 'FIN_AMOUNT_WOE',
 'due_date_WOE',
 'last_Month_coll_WOE',
 'nuof_EMI_L3M_WOE',
 'nuof_EMI_L6M_WOE',
 'nuof_EMI_L9M_WOE',
 'MaxBkt_12M_WOE',
 'CountInBkt_3_3M_WOE',
 'CountInBkt_1Plus_3M_WOE',
 'CountInBkt_1Plus_6M_WOE',
 'CountInBkt_2Plus_3M_WOE',
 'CountInBkt_2Plus_6M_WOE',
 'CountInBkt_3Plus_12M_WOE',
 'CountInBkt_3Plus_24M_WOE',
 'RollFw1_WOE',
 'RollFw2_WOE',
 'RollFw3_WOE',
 'RollFw6_WOE',
 'Roll_stable_2_WOE',
 'Roll_stable_3_WOE',
 'CE_WOE',
 'TENURE1_WOE'
]

In [ ]:
df['MONTH'].unique()

In [ ]:
# Target Columns
selected__target_column ='TARGET_VARIABLE_V1'

# Key Columns
drop_columns1=['CONNO','BPNO','ODBKT','SOHPOD','SOHPOD_PREASSIGN']



# Selected columns for separate train and test data
# month='NONE' # Don't want to select train and test data based on month
month='MONTH'


month_column=month

# Selected months for train data
train_months=[
'JAN24', 'FEB24', 'MAR24', 'APR24', 'MAY24', 'JUN24','JUL24', 'AUG24', 'SEP24',
        'OCT24', 'NOV24', 'DEC24'
]

test_months=[
'JAN25', 'FEB25',
       'MAR25'
]


# Select Models diff_models=['Logistic Regression', 'Random Forest', 'Decision Tree', 'XGBoost', 'LightGBM', 'CatBoost']
diff_models = ['Logistic Regression', 'Random Forest', 'Decision Tree', 'XGBoost', 'LightGBM', 'CatBoost']
models_prev = 'Random Forest'
models = 'Random Forest'

  # Model Building Steps

In [ ]:
# if models_prev=="Logistic Regression":
#     model_prev = LogisticRegression(random_state=42)
#     pass
# elif models_prev=="Random Forest":
#     model_prev=RandomForestClassifier(n_estimators=500, random_state=42) 
#     pass
# elif models_prev=="Decision Tree":
#     model_prev=DecisionTreeClassifier(random_state=42)
#     pass
# elif models_prev=="XGBoost":
#     model_prev=xgb.XGBClassifier(n_estimators=500, random_state=42)
#     pass
# elif models_prev=="LightGBM":
#     model_prev=lgb.LGBMClassifier(n_estimators=500, random_state=42)
#     pass
# elif models_prev=="CatBoost":
#     model_prev=CatBoostClassifier(iterations=500, random_state=42, verbose=0)
#     pass
# ##############################################################################3

# if models=="Logistic Regression":
#     model = LogisticRegression(random_state=42)
#     pass
# elif models=="Random Forest":
#     model=RandomForestClassifier(n_estimators=500, random_state=42) 
#     pass
# elif models=="Decision Tree":
#     model=DecisionTreeClassifier(random_state=42)
#     pass
# elif models=="XGBoost":
#     model=xgb.XGBClassifier(n_estimators=500, random_state=42)
#     pass
# elif models=="LightGBM":
#     model=lgb.LGBMClassifier(n_estimators=500, random_state=42)
#     pass
# elif models=="CatBoost":
#     model=CatBoostClassifier(iterations=500, random_state=42, verbose=0)
#     pass
# ##############################################################################3

# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#     #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# # Model Building steps started


# # Model used Variables
# drop_columns=drop_columns1+[selected__target_column]+[month]
# other_columns=final_variables_list

#     # st.write(drop_columns1)
#     # st.write(other_columns)

#     # required columns

    

# # Data 
# df=df[drop_columns+other_columns]
# DATA=df

# if month=="NONE":
#     train_data=df
# else:
#     train_data = df[df[month].isin(train_months)]


# X_train = train_data.drop(drop_columns, axis=1)
# y_train = train_data[selected__target_column]
# model_prev.fit(X_train, y_train)

# # # Loop through each test month and evaluate the model
#     # for months in test_months:
        
#     #     test_data = DATA[DATA[month] == months]
        

#     #     # Separate features (X) and target (y) for testing data
#     #     X_test = test_data.drop(drop_columns, axis=1)
#     #     y_test = test_data[selected__target_column]

#     #     # Apply the optimal threshold to the test set
#     #     y_pred = model.predict_proba(X_test)[:, 1]

#     # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#     # Check Feature Importance and multicolinearity for Logistic Regression

# if model_prev=="Logistic Regression":
#     # Calculate the Multicolinearity and select the variables like feature importance
#     # Calculate VIF for each feature
#     importance_df = pd.DataFrame()
    

#     importance_df["Feature"] = X_train.columns
#     importance_df["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
#     importance_df = importance_df.sort_values("VIF", ascending=True)
#     importance_df = importance_df.reset_index(drop=True)
#     features=importance_df['Feature'].tolist()

#     # importance_df

    

# else:



#     # Get feature importances
#     feature_importances = model_prev.feature_importances_
#     # features = columns
#     features = X_train.columns.to_list()

#     # Create a DataFrame to display feature importances
#     importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances.round(5)})
#     importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index()

# importance_df

    
    


# Cross Validation

In [ ]:




# Define param grids
param_grids = {
    "Logistic Regression": {
        'C': [0.01, 0.1, 1.0, 10],
        'solver': ['lbfgs'],
        'max_iter': [100, 200]
    },
    "Random Forest": {
        'n_estimators': [100, 300, 500],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5]
    },
    "Decision Tree": {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10]
    },
    "XGBoost": {
        'n_estimators': [100, 300, 500],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    "LightGBM": {
        'n_estimators': [100, 300, 500],
        'max_depth': [3, 5, 7, -1],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    "CatBoost": {
        'iterations': [100, 300, 500],
        'depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'verbose': [0]
    }
}

# Define base models
base_models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42),
    "LightGBM": lgb.LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0)
}

# Model used Variables
drop_columns=drop_columns1+[selected__target_column]+[month]
other_columns=final_variables_list
# Prepare data
df = df[drop_columns + other_columns]
DATA = df
train_data = df if month == "NONE" else df[df[month].isin(train_months)]

X_train = train_data.drop(drop_columns, axis=1)
y_train = train_data[selected__target_column]

# Perform Grid Search CV for models_prev
grid_prev = GridSearchCV(estimator=base_models[models_prev], 
                         param_grid=param_grids[models_prev], 
                         cv=3, scoring='roc_auc', n_jobs=-1,verbose=2)
grid_prev.fit(X_train, y_train)
model_prev = grid_prev.best_estimator_

# Perform Grid Search CV for models (main model to be used)
grid_main = GridSearchCV(estimator=base_models[models], 
                         param_grid=param_grids[models], 
                         cv=3, scoring='roc_auc', n_jobs=-1,verbose=2)
grid_main.fit(X_train, y_train)
model = grid_main.best_estimator_




In [ ]:
# Feature Importance / VIF
if models_prev == "Logistic Regression":
    # Optional: Scale data for VIF
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

    importance_df = pd.DataFrame()
    importance_df["Feature"] = X_train.columns
    importance_df["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_train.shape[1])]
    importance_df = importance_df.sort_values("VIF", ascending=True).reset_index(drop=True)
    features = importance_df['Feature'].tolist()

else:
    feature_importances = model_prev.feature_importances_
    features = X_train.columns.to_list()
    importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances.round(5)})
    importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)



In [ ]:
model_prev

In [ ]:
model

In [ ]:
# Show importance
importance_df

# Top 10 features

In [ ]:


number_of_features=20

top_features = importance_df.head(number_of_features)['Feature'].values.tolist()

# Get the indices of the top features in the original feature set
top_features_indices = [features.index(f) for f in top_features]

top_features



# Filter columns from top features

In [ ]:
top_features=[
'CountInBkt_3_3M_WOE',
 'Roll_stable_2_WOE',
 'due_date_WOE',
 'RollFw2_WOE',
 'last_Month_coll_WOE',
#  'Roll_stable_3_WOE',
 'CountInBkt_2Plus_3M_WOE',
#  'CountInBkt_2Plus_6M_WOE',
 'nuof_EMI_L3M_WOE',
#  'RollFw1_WOE',
#  'RollFw6_WOE',
#  'CountInBkt_1Plus_6M_WOE',
#  'RollFw3_WOE',
 'DSA_DLR_WOE',
 'BRANCH_WOE',
#  'nuof_EMI_L6M_WOE',
 'CE_WOE',
#  'CountInBkt_1Plus_3M_WOE',
#  'CountInBkt_3Plus_12M_WOE',
 'PRODUCT_New_WOE'
    
]

# Check Multicolinearity

In [ ]:
importance_df = pd.DataFrame()
    

importance_df["Feature"] = top_features
importance_df["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(len(top_features))]
importance_df = importance_df.sort_values("VIF", ascending=True)
importance_df = importance_df.reset_index(drop=True)
features=importance_df['Feature'].tolist()

importance_df

# Select Final Features based on Multicolinearity

In [ ]:


number_of_features=10

top_features = importance_df.head(number_of_features)['Feature'].values.tolist()

# Get the indices of the top features in the original feature set
top_features_indices = [features.index(f) for f in top_features]

top_features



# Check Variable importance Again

In [ ]:

X_train1 = X_train[top_features]
y_train1 = train_data[selected__target_column]
model_prev.fit(X_train1, y_train1)



# Check Feature Importance and multicolinearity for Logistic Regression

if model_prev=="Logistic Regression":
    # Calculate the Multicolinearity and select the variables like feature importance
    # Calculate VIF for each feature
    importance_df = pd.DataFrame()
    

    importance_df["Feature"] = top_features
    importance_df["VIF"] = [variance_inflation_factor(X_train1.values, i) for i in range(top_features)]
    importance_df = importance_df.sort_values("VIF", ascending=True)
    importance_df = importance_df.reset_index(drop=True)
    features=importance_df['Feature'].tolist()

    # importance_df

    

else:



    # Get feature importances
    feature_importances = model_prev.feature_importances_
    # features = columns
    features = top_features

    # Create a DataFrame to display feature importances
    importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances.round(5)})
    importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index()

importance_df

    

# Select Final Features based on Variable Importance Again

In [ ]:


number_of_features=10

top_features = importance_df.head(number_of_features)['Feature'].values.tolist()

# Get the indices of the top features in the original feature set
top_features_indices = [features.index(f) for f in top_features]

top_features



In [ ]:
# top_features=[
# 'CountInBkt_3_3M_WOE',
#  'due_date_WOE',
#  'Roll_stable_2_WOE',
#  'nuof_EMI_L3M_WOE',
#  'MaxBkt_12M_WOE',
#  'CountInBkt_2Plus_6M_WOE',
#  'DSA_DLR_WOE',
# #  'Restructuring_Flag_WOE',
#  'last_Month_coll_WOE',
#  'nuof_EMI_L6M_WOE',
#  'PRODUCT_New_WOE',
# #  'Ever_90_WOE',
#  'BRANCH_WOE',
#  'RollFw3_WOE',
# #  'max_odbkt_WOE'

# ]

In [ ]:

#############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Define the ks_table function
def ks_table(data_frame, pred_col_name='Probability', true_col_name='Target', responder_default=1, dec_no=10):
    ascending_df = data_frame.sort_values(by=pred_col_name, ascending=False)
    length_df = len(data_frame)
    decile_len = length_df // dec_no
    total_resp = data_frame[true_col_name].value_counts().loc[1]
    total_non_resp = data_frame[true_col_name].value_counts().loc[0]
    temp_ks = pd.DataFrame({
        'Decile': [i + 1 for i in range(dec_no)],
        'Total Population': [decile_len] * dec_no,
        'Max Probability': [np.nan] * dec_no,
        'Min Probability': [np.nan] * dec_no,
        'Good': [np.nan] * dec_no,
        'Bad': [np.nan] * dec_no,
        'Good %': [np.nan] * dec_no,
        'Bad %': [np.nan] * dec_no,
        'Cumulative Good %': [np.nan] * dec_no,
        'Cumulative Bad %': [np.nan] * dec_no,
        'Difference': [np.nan] * dec_no
    })
    cum_prec_responders = 0
    cum_prec_non_responders = 0
    for j in range(dec_no):
        if j < dec_no - 1:
            temp_df = ascending_df.iloc[decile_len * j:decile_len * (j + 1)]
        else:
            temp_df = ascending_df.iloc[decile_len * j:]
        min_prob = min(temp_df[pred_col_name])
        max_prob = max(temp_df[pred_col_name])
        try:
            non_responders = temp_df[true_col_name].value_counts().loc[0]
        except KeyError:
            non_responders = 0
        try:
            responders = temp_df[true_col_name].value_counts().loc[1]
        except KeyError:
            responders = 0
        prec_responders = responders / total_resp
        prec_non_responders = non_responders / total_non_resp
        cum_prec_responders = prec_responders + cum_prec_responders
        cum_prec_non_responders = prec_non_responders + cum_prec_non_responders
        diff = cum_prec_responders - cum_prec_non_responders
        temp_ks.loc[j, 'Max Probability'] = format(max_prob, 'f')
        temp_ks.loc[j, 'Min Probability'] = format(min_prob, 'f')
        temp_ks.loc[j, 'Good'] = int(non_responders)
        temp_ks.loc[j, 'Bad'] = int(responders)
        temp_ks.loc[j, 'Good %'] = prec_non_responders
        temp_ks.loc[j, 'Bad %'] = prec_responders
        temp_ks.loc[j, 'Cumulative Good %'] = cum_prec_non_responders
        temp_ks.loc[j, 'Cumulative Bad %'] = cum_prec_responders
        temp_ks.loc[j, 'Difference'] = format(diff, 'f')

    return temp_ks

# Define the ks_table_TEST function
def ks_table_TEST(data_frame, pred_col_name='Probability', true_col_name='Target', responder_default=1, dec_no=10, min_max_ranges=None):
    if min_max_ranges is None or len(min_max_ranges) != dec_no:
        raise ValueError("min_max_ranges must be provided and should have a length equal to dec_no")

    # Sort the DataFrame by the prediction probabilities in descending order
    ascending_df = data_frame.sort_values(by=pred_col_name, ascending=False)
    
    # Prepare the KS table DataFrame
    temp_ks = pd.DataFrame({
        'Decile': [i + 1 for i in range(dec_no)],
        'Total Population': [np.nan] * dec_no,
        'Max Probability': [np.nan] * dec_no,
        'Min Probability': [np.nan] * dec_no,
        'Good': [np.nan] * dec_no,
        'Bad': [np.nan] * dec_no,
        'Good %': [np.nan] * dec_no,
        'Bad %': [np.nan] * dec_no,
        'Cumulative Good %': [np.nan] * dec_no,
        'Cumulative Bad %': [np.nan] * dec_no,
        'Difference': [np.nan] * dec_no
    })
    
    total_resp = data_frame[true_col_name].value_counts().get(1, 0)
    total_non_resp = data_frame[true_col_name].value_counts().get(0, 0)
    
    cum_prec_responders = 0
    cum_prec_non_responders = 0

    for j in range(dec_no):
        min_prob, max_prob = min_max_ranges[j]
        
        # Filter rows in the current decile range
        temp_df = ascending_df[(ascending_df[pred_col_name] >= min_prob) & (ascending_df[pred_col_name] <= max_prob)]
        
        total_pop = len(temp_df)
        non_responders = temp_df[true_col_name].value_counts().get(0, 0)
        responders = temp_df[true_col_name].value_counts().get(1, 0)
        
        prec_responders = responders / total_resp if total_resp != 0 else 0
        prec_non_responders = non_responders / total_non_resp if total_non_resp != 0 else 0
        
        cum_prec_responders += prec_responders
        cum_prec_non_responders += prec_non_responders
        
        diff = cum_prec_responders - cum_prec_non_responders
        
        temp_ks.loc[j, 'Total Population'] = total_pop
        temp_ks.loc[j, 'Max Probability'] = format(max_prob, 'f')
        temp_ks.loc[j, 'Min Probability'] = format(min_prob, 'f')
        temp_ks.loc[j, 'Good'] = int(non_responders)
        temp_ks.loc[j, 'Bad'] = int(responders)
        temp_ks.loc[j, 'Good %'] = prec_non_responders
        temp_ks.loc[j, 'Bad %'] = prec_responders
        temp_ks.loc[j, 'Cumulative Good %'] = cum_prec_non_responders
        temp_ks.loc[j, 'Cumulative Bad %'] = cum_prec_responders
        temp_ks.loc[j, 'Difference'] = format(diff, 'f')
    
    return temp_ks

In [ ]:


if month=="NONE":

    # # Separate features (X) and target (y) for training data
    X_train = train_data.drop(drop_columns, axis=1)
    # y_train = train_data['TARGET']

    # X_train=X_train_selected
    # X_train=X_train.iloc[:, top_features_indices]
    X_train=X_train[top_features]
    y_train=train_data[selected__target_column]




    # Fit the model
    model.fit(X_train,y_train)

    # Calculate the optimal threshold based on ROC curve for training data
    y_prob_cutoff_train = model.predict_proba(X_train)[:, 1]
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_prob_cutoff_train)
    optimal_idx_train = np.argmax(tpr_train - fpr_train)
    optimal_threshold = thresholds_train[optimal_idx_train]

    y_pred_train = np.where(y_prob_cutoff_train >= optimal_threshold, 1, 0)

    # Print evaluation metrics for training data
    cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
    disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=model.classes_)
    disp_train.plot()
    # plt.show()

    # Calculate KS table for training data
    predictions_train = pd.DataFrame({'Target': y_train, 'Probability': y_prob_cutoff_train})
    ks_table_train = ks_table(predictions_train, pred_col_name='Probability', true_col_name='Target', responder_default=1, dec_no=10)



    # Add columns to Train data

    # Add the required columns to train_data
    train_data['Probability'] = y_prob_cutoff_train

    # Step: Add Decile column to train_data based on ks_table_train ranges

    def assign_decile(prob, ks_table_df):
        for i, row in ks_table_df.iterrows():
            if float(row['Min Probability']) <= prob <= float(row['Max Probability']):
                return row['Decile']
        return np.nan  # if not matched (optional safeguard)

    train_data['Decile'] = train_data['Probability'].apply(lambda x: assign_decile(x, ks_table_train))



    # Dictionary to store KS tables and concatenated DataFrames for each test month
    ks_tables_test = {}
    decile_dfs = {}

    # all data with top features

    all_data=[train_data]

    final_data = pd.concat(all_data, ignore_index=True)
    required_columnss=drop_columns1+[month_column]+[selected__target_column]+top_features+['Decile','Probability']
    final_data=final_data[required_columnss]


else:
    # # Separate features (X) and target (y) for training data
    X_train = train_data.drop(drop_columns, axis=1)
    # y_train = train_data['TARGET']

    # X_train=X_train_selected
    # X_train=X_train.iloc[:, top_features_indices]
    X_train=X_train[top_features]
    y_train=train_data[selected__target_column]




    # Fit the model
    model.fit(X_train,y_train)

    # Calculate the optimal threshold based on ROC curve for training data
    y_prob_cutoff_train = model.predict_proba(X_train)[:, 1]
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_prob_cutoff_train)
    optimal_idx_train = np.argmax(tpr_train - fpr_train)
    optimal_threshold = thresholds_train[optimal_idx_train]

    y_pred_train = np.where(y_prob_cutoff_train >= optimal_threshold, 1, 0)

    # Print evaluation metrics for training data
    # cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
    # disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=model.classes_)
    # disp_train.plot()
    # plt.show()

    # Calculate KS table for training data
    predictions_train = pd.DataFrame({'Target': y_train, 'Probability': y_prob_cutoff_train})
    ks_table_train = ks_table(predictions_train, pred_col_name='Probability', true_col_name='Target', responder_default=1, dec_no=10)



    # Add columns to Train data

    # Add the required columns to train_data
    train_data['Probability'] = y_prob_cutoff_train

    # Step: Add Decile column to train_data based on ks_table_train ranges

    def assign_decile(prob, ks_table_df):
        for i, row in ks_table_df.iterrows():
            if float(row['Min Probability']) <= prob <= float(row['Max Probability']):
                return row['Decile']
        return np.nan  # if not matched (optional safeguard)

    train_data['Decile'] = train_data['Probability'].apply(lambda x: assign_decile(x, ks_table_train))



    # Dictionary to store KS tables and concatenated DataFrames for each test month
    ks_tables_test = {}
    decile_dfs = {}

    # all data with top features

    all_data=[train_data]

    # Loop through each test month and evaluate the model
    for months in test_months:
        

        test_data = DATA[DATA[month_column] == months]

        # Separate features (X) and target (y) for testing data
        X_test = test_data.drop(drop_columns, axis=1)
        # X_test=X_test_selected
        X_test=X_test[top_features]
        y_test = test_data[selected__target_column]

        # Apply the optimal threshold to the test set
        y_prob_cutoff_test = model.predict_proba(X_test)[:, 1]
        y_prob_test_with_cutoff = np.where(y_prob_cutoff_test >= optimal_threshold, 1, 0)



            # Add the required columns to test_data

        test_data['Probability'] = y_prob_cutoff_test

        # Step: Add Decile column to train_data based on ks_table_train ranges

        def assign_decile(prob, ks_table_df):
            for i, row in ks_table_df.iterrows():
                if float(row['Min Probability']) <= prob <= float(row['Max Probability']):
                    return row['Decile']
            return np.nan  # if not matched (optional safeguard)

        test_data['Decile'] = test_data['Probability'].apply(lambda x: assign_decile(x, ks_table_train))

            

        # Plot the confusion matrix
        # cm = confusion_matrix(y_test, y_prob_test_with_cutoff, labels=model.classes_)
        # disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
        # disp.plot()
        # plt.show()

        # Calculate and store KS table in dictionary with DataFrame name including month
        predictions_val = pd.DataFrame({'Target': y_test, 'Probability': y_prob_cutoff_test})
        min_max_ranges = [(np.float32(row['Min Probability']), np.float32(row['Max Probability'])) for _, row in ks_table_train.iterrows()]
        ks_table_test = ks_table_TEST(predictions_val, pred_col_name='Probability', true_col_name='Target', responder_default=1, dec_no=10, min_max_ranges=min_max_ranges)
        ks_tables_test[f"KS_TABLE_{months}"] = ks_table_test



            # Append the test data for the month to the all_data list
        all_data.append(test_data)

    # Concatenate all the data (train and test data) into a single DataFrame
    final_data = pd.concat(all_data, ignore_index=True)
    required_columnss=drop_columns1+[month_column]+[selected__target_column]+top_features+['Decile','Probability']
    final_data=final_data[required_columnss]


# Train Model Results

In [ ]:
acc = accuracy_score(y_train, y_pred_train)
f1 = f1_score(y_train, y_pred_train)
recall = recall_score(y_train, y_pred_train)
bal_acc = balanced_accuracy_score(y_train, y_pred_train)
precision= precision_score(y_train, y_pred_train)
roc_auc = roc_auc_score(y_train, y_pred_train)



print("Accuracy", f"{acc:.4f}")
print("Precision", f"{precision:.4f}")
print("F1 Score", f"{f1:.4f}")
print("Recall", f"{recall:.4f}")
print("Balanced Accuracy", f"{bal_acc:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")



# Classification Report
print("##################################################################")
print("**Classification Report:**")
print(classification_report(y_train, y_pred_train))

print("##################################################################")


# Confusion Matrix
print("**Confusion Matrix:**")
cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=model.classes_)

# disp_train.plot()
# plt.show()
print("##################################################################")
fig, ax = plt.subplots()
disp_train.plot(ax=ax)
st.pyplot(fig)
plt.show()
print("##################################################################")


# Test Model Results

# Comparision Table

In [ ]:
# Comparison table
# Sample data for development and validation (replace with actual data)
development_data = pd.DataFrame({
    'Decile':[1,2,3,4,5,6,7,8,9,10],
    'Bad': ks_table_train['Bad'],
    'Good': ks_table_train['Good']
})

validation_data = {
    month: pd.DataFrame({
        'Bad': ks_tables_test[f"KS_TABLE_{month}"]['Bad'],
        'Good': ks_tables_test[f"KS_TABLE_{month}"]['Good']
    })
    for month in test_months
}

# validation_data

# Helper function to calculate cumulative and derived values
def calculate_metrics(data):
    total_bad = data['Bad'].sum()
    total_good = data['Good'].sum()

    data['Total'] = data['Bad'] + data['Good']
    data['Cumulative Bad'] = data['Bad'].cumsum()
    data['Cumulative Good'] = data['Good'].cumsum()
    data['Cumulative Bad%'] = data['Cumulative Bad'] / total_bad
    data['Cumulative Good%'] = data['Cumulative Good'] / total_good
    data['Bad Rate'] = data['Bad'] / data['Total']
    data['%Bad'] = data['Bad'] / total_bad
    data['%Good'] = data['Good'] / total_good
    data['KS'] = data['Cumulative Bad%']-data['Cumulative Good%']
    # Add an 'AUC' column to the DataFrame for transparency
    data['AUC'] =(data['Cumulative Good%'] / 2) * (data['Cumulative Bad%'])
    for i in range(1, len(data)):
        data.at[i, 'AUC'] = ((data['Cumulative Good%'][i] + data['Cumulative Good%'][i-1]) / 2) * (data['Cumulative Bad%'][i] - data['Cumulative Bad%'][i-1])
    data['Rank'] = data['Bad Rate'].rank(ascending=False)
    data['Gini'] =''


    return data, total_bad, total_good

# Calculate for development data
dev_data, dev_total_bad, dev_total_good = calculate_metrics(development_data)

# Calculate for validation data (for each month)
validation_metrics = {}
for month, val_data in validation_data.items():
    val_data, _, _ = calculate_metrics(val_data)
    validation_metrics[month] = val_data



# # Calculate ROB (Rank of Best) for each month
for month, val_data in validation_metrics.items():
    val_data['ROB'] = (dev_data['Rank'] != val_data['Rank']).astype(int)

# # Calculate ROB (Rank of Best) for each month
for month, val_data in validation_metrics.items():
    val_data['Gini'] = ''


# Combine Development Set and Validation Sets side by side
combined_data = dev_data.copy()
for month, val_data in validation_metrics.items():
    # Align the validation data with the development data by index (assuming same number of rows)
    val_data = val_data.rename(columns=lambda x: f"{x} ({month})")  # Rename columns to distinguish months
    combined_data = pd.concat([combined_data, val_data], axis=1)


# Add an overall total row at the end
overall_totals = pd.DataFrame({
        'Decile': ['Overall'],
        'Bad': combined_data['Bad'].sum(axis=0),
        'Good': combined_data['Good'].sum(axis=0),
        'Total': combined_data['Total'].sum(axis=0),
        'Bad Rate': combined_data['Bad'].sum(axis=0)/combined_data['Total'].sum(axis=0),
        '%Bad': combined_data['%Bad'].sum(axis=0),
        '%Good': combined_data['%Good'].sum(axis=0),
        'KS': combined_data['KS'].max(),
        'AUC': combined_data['AUC'].sum(axis=0),
        'Gini': abs(1 - 2 * combined_data['AUC'].sum(axis=0)),

        **{f"Bad ({month})": combined_data[f"Bad ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"Good ({month})": combined_data[f"Good ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"Total ({month})": combined_data[f"Total ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"Bad Rate ({month})": combined_data[f"Bad ({month})"].sum(axis=0)/combined_data[f"Total ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"%Bad ({month})": combined_data[f"%Bad ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"%Good ({month})": combined_data[f"%Good ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"KS ({month})": combined_data[f"KS ({month})"].max() for month in validation_metrics},
        **{f"AUC ({month})": combined_data[f"AUC ({month})"].sum(axis=0) for month in validation_metrics},
        **{f"ROB ({month})": combined_data[f"ROB ({month})"].sum(axis=0)/combined_data[f"ROB ({month})"].count() for month in validation_metrics},
        **{f"Gini ({month})": (abs(1-2*(combined_data[f"AUC ({month})"].sum(axis=0)))) for month in validation_metrics}
})

# Append the overall totals row at the end of the combined data
combined_data_with_totals = pd.concat([combined_data, overall_totals], ignore_index=True)

# Display the final combined data with the overall totals row
combined_data_with_totals


In [ ]:
cumulative_bad_columns = list(combined_data_with_totals.columns[(combined_data_with_totals.columns.str.startswith("Cumulative Bad%"))|(combined_data_with_totals.columns.str.startswith("Bad Rate"))|(combined_data_with_totals.columns.str.startswith("ROB"))])
# bad_rate_columns=combined_data_with_totals.columns[combined_data_with_totals.columns.str.startswith("Bad Rate")]

# cumulative_bad_columns=['Decile']+cumulative_bad_columns
# print(cumulative_bad_columns)
combined_data_with_totals_V1=combined_data_with_totals[["Decile"]+cumulative_bad_columns]

combined_data_with_totals_V1

# PSI table

In [ ]:
# Decile name

Decile='Decile'



def calculate_psi_multiple_tests(train_df, test_dfs, total_pop_col_train, total_pop_cols_test):
    """
    Calculate PSI for multiple test datasets compared to a single train dataset.

    Parameters:
    - train_df: DataFrame for the train KS table.
    - test_dfs: Dictionary of DataFrames for the test KS tables, where keys are test month labels.
    - total_pop_col_train: Column name for total population in the train table.
    - total_pop_cols_test: Dictionary where keys are test month labels, and values are column names for total population in each test table.

    Returns:
    - psi_table: Combined PSI table for all test months.

    """

    # Validate matching keys
    missing_keys = [month for month in test_dfs if month not in total_pop_cols_test]
    if missing_keys:
        raise KeyError(f"Missing keys in total_pop_cols_test for months: {missing_keys}")
    # Ensure the train DataFrame is sorted by decile
    train_df = train_df.sort_values(by=Decile).reset_index(drop=True)
    
    
    # Calculate total train population and population percentage per decile
    total_train_pop = train_df[total_pop_col_train].sum()
    train_df['Train Population %'] = train_df[total_pop_col_train] / total_train_pop
    
    # Initialize the PSI table with train data columns
    psi_table = pd.DataFrame({
        Decile: train_df[Decile],
        'Train Total Population': train_df[total_pop_col_train],
        'Train Population %': train_df['Train Population %']
    })
    
    # Dictionary to store total PSI values for each month
    total_psi_values = {}

    # Calculate PSI for each test month
    for month, test_df in test_dfs.items():
        # st.write("**`Test Month:`**",month)
        # st.write(test_df)
        # st.write(test_dfs)
        # st.write(total_pop_cols_test)
        test_df = test_df.sort_values(by=Decile).reset_index(drop=True)

        
        
        # Calculate total test population and population percentage per decile
        total_test_pop = test_df[total_pop_cols_test[month]].sum()
        test_df['Test Population %'] = test_df[total_pop_cols_test[month]] / total_test_pop
        
        # Calculate PSI for each decile in this test month
        psi_values = (psi_table['Train Population %'] - test_df['Test Population %']) * \
                    np.log(psi_table['Train Population %'] / test_df['Test Population %'])
        
        # Add columns for the test month's population and PSI values
        psi_table[f'{month} Test Total Population'] = test_df[total_pop_cols_test[month]]
        psi_table[f'{month} Test Population %'] = test_df['Test Population %']
        psi_table[f'{month} PSI'] = psi_values
        
        # Store the total PSI for this month
        total_psi_values[month] = psi_values.sum()
    
    # Calculate overall decision based on maximum PSI across all months
    max_total_psi = max(total_psi_values.values())
    if max_total_psi < 0.1:
        psi_decision = 'Stable'
    elif max_total_psi < 0.25:
        psi_decision = 'On Watch'
    else:
        psi_decision = 'Unstable'
    
    # Create a single Total row with the sum of populations and percentages
    total_row = pd.DataFrame({
        Decile: ['Total'],
        'Train Total Population': [train_df[total_pop_col_train].sum()],
        'Train Population %': [train_df['Train Population %'].sum()],
        **{f'{month} Test Total Population': [test_dfs[month][total_pop_cols_test[month]].sum()] for month in test_dfs},
        **{f'{month} Test Population %': [psi_table[f'{month} Test Population %'].sum()] for month in test_dfs},
        **{f'{month} PSI': [total_psi_values[month]] for month in test_dfs}
    })

    # Create a single Decision row with the PSI decision
    decision_row = pd.DataFrame({
        Decile: ['Decision'],
        'Train Total Population': [''],
        'Train Population %': [''],
        **{f'{month} Test Total Population': [''] for month in test_dfs},
        **{f'{month} Test Population %': [''] for month in test_dfs},
        **{f'{month} PSI': [psi_decision] for month in test_dfs}
    })
    
    # Concatenate the PSI table with the single Total and Decision rows
    psi_table = pd.concat([psi_table, total_row, decision_row], ignore_index=True)
    return psi_table


# Automatically extract short month labels (e.g., "Jan" from "JAN25")
def get_short_month(month_code):
    return month_code[:3].capitalize()  # 'JAN' → 'Jan'


# Build train_df dynamically
train_df = ks_table_train

# # Automatically extract short month labels (e.g., "Jan" from "JAN25")
# def get_short_month(month_code):
#     return month_code[:3].capitalize()  # 'JAN' → 'Jan'



# Build test_dfs and total_pop_cols_test dynamically
test_dfs = {
    monthss: ks_tables_test[f"KS_TABLE_{monthss}"]
    for monthss in test_months
}
# test_dfs

# st.write(test_dfs)

total_pop_cols_test = {
    monthss: 'Total Population'
    for monthss in test_months
}
# total_pop_cols_test

# Calculate PSI for multiple test datasets compared to a single train dataset and View test data

# with st.expander("**`View PSI table`**",icon="📅"):
psi_table = calculate_psi_multiple_tests(train_df, test_dfs, 'Total Population', total_pop_cols_test)

# Display the PSI table
psi_table

# KS table train

In [ ]:
ks_table_train

# KS TEST

In [ ]:
for  ks_table_test1 in test_months:

    ks_table_test2=ks_tables_test[f"KS_TABLE_{ks_table_test1}"]
    # st.write(ks_table_test2)
    # Display KS table for selected test month
    print("#######################################################################")
    print(f"**`KS Table for {ks_table_test1}:`**")
    print(ks_table_test2)
    print("#######################################################################")


### Train Data

In [ ]:
# # Train evaluation metrics
# train_evaluation_df = pd.DataFrame({
#     'Model Name': [model],
#     'Target_Variable': [selected__target_column],
#     'Optimal_Threshold': [optimal_threshold],
#     'ROC_AUC SCORE': [roc_auc],
#     'Accuracy': [accuracy_score(y_train, y_pred_train)],
#     'Precision': [precision_score(y_train, y_pred_train)],
#     'Recall': [recall_score(y_train, y_pred_train)],
#     'F1_Score': [f1_score(y_train, y_pred_train)],
#     'Balanced_Accuracy': [balanced_accuracy_score(y_train, y_pred_train)],
#     'Variable Used in model':[top_features]
# })

# # ----------------------
# # 2. Create Train Confusion Matrix DataFrame
# # ----------------------

# cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
# train_confusion_matrix_df = pd.DataFrame(
#     cm_train,
#     index=[f"Actual_{cls}" for cls in model.classes_],
#     columns=[f"Predicted_{cls}" for cls in model.classes_]
# )


# # ----------------------
# # 4. Prepare KS Tables (train and tests)
# # ----------------------

# # Train KS Table
# train_ks_table = ks_table_train



In [ ]:
###
# Train evaluation metrics
train_evaluation_df = pd.DataFrame({
    'Model Name': [model],
    'Target_Variable': ['TARGET'],
    'Optimal_Threshold': [optimal_threshold],
    'Accuracy': [accuracy_score(y_train, y_pred_train)],
    'Precision': [precision_score(y_train, y_pred_train)],
    'Recall': [recall_score(y_train, y_pred_train)],
    'F1_Score': [f1_score(y_train, y_pred_train)],
    'Balanced_Accuracy': [balanced_accuracy_score(y_train, y_pred_train)],
    'Variable Used in model':[top_features]
})

# ----------------------
# 2. Create Train Confusion Matrix DataFrame
# ----------------------

cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
train_confusion_matrix_df = pd.DataFrame(
    cm_train,
    index=[f"Actual_{cls}" for cls in model.classes_],
    columns=[f"Predicted_{cls}" for cls in model.classes_]
)

# ----------------------
# 3. Create Test Evaluation DataFrames
# ----------------------

test_evaluation_dfs = {}
# st.write(month_column)



for months in test_months:
    test_data = DATA[DATA[month_column] == months]
    
    X_test = test_data.drop(drop_columns, axis=1)
    X_test = X_test[top_features]
    y_test = test_data[selected__target_column]
    
    y_prob_cutoff_test = model.predict_proba(X_test)[:, 1]
    y_pred_test = np.where(y_prob_cutoff_test >= optimal_threshold, 1, 0)

    test_evaluation_df = pd.DataFrame({
        'Target_Variable': [selected__target_column],
        'Month': [months],
        'Optimal_Threshold': [optimal_threshold],
        'Accuracy': [accuracy_score(y_test, y_pred_test)],
        'Precision': [precision_score(y_test, y_pred_test)],
        'Recall': [recall_score(y_test, y_pred_test)],
        'F1_Score': [f1_score(y_test, y_pred_test)],
        'Balanced_Accuracy': [balanced_accuracy_score(y_test, y_pred_test)]
    })
    
    test_evaluation_dfs[months] = test_evaluation_df


    confusion_test= confusion_matrix(y_test, y_pred_test, labels=model.classes_)
    test_confusion_matrix_df = pd.DataFrame(
        confusion_test,index=[f"Actual_{cls}" for cls in model.classes_],
        columns=[f"Predicted_{cls}" for cls in model.classes_])

# ----------------------
# 4. Prepare KS Tables (train and tests)
# ----------------------

# Train KS Table
train_ks_table = ks_table_train

# Test KS Tables
test_ks_tables = {}
for month in test_months:
    test_ks_tables[month] = ks_tables_test[f"KS_TABLE_{month}"]

# Export File

In [ ]:
# Clean name1 and model
name1_clean = file_name_main.replace(" ", "_")
model_clean = models.replace(" ", "_")



# Final CSV
final_data_csv = final_data.to_csv(f"{path_main}\{name1_clean}_final_data_with_probability.csv",index=False)
    
# Create Excel with multiple sheets
excel_buffer = io.BytesIO()
with pd.ExcelWriter(rf"{path_main}\{name1_clean}_{model_clean}_model_summary.xlsx", engine='xlsxwriter') as writer:

    #PSI table
    psi_table.to_excel(writer, sheet_name='PSI_Table', index=False)
    # Comparision Table
    combined_data_with_totals.to_excel(writer, sheet_name='Comparison_Table', index=False)

    # KS Tables
    train_ks_table.to_excel(writer, sheet_name="Train_KS_Table", index=False)
    for month, df in test_ks_tables.items():
        safe_month = month.replace("/", "-").replace(" ", "_")
        df.to_excel(writer, sheet_name=f"Test_KS_{safe_month}", index=False)

    # Train Results
    train_evaluation_df.to_excel(writer, sheet_name='Train_Evaluation', index=False)
    train_confusion_matrix_df.to_excel(writer, sheet_name='Train_Confusion_Matrix', index=True)

    # Test Results
    for month, df in test_evaluation_dfs.items():
        safe_month = month.replace("/", "-").replace(" ", "_")  # safe sheet name
        df.to_excel(writer, sheet_name=f"Test_Evaluation_{safe_month}", index=False)
        test_confusion_matrix_df.to_excel(writer, sheet_name=f"Test_Confusion_Matrix_{safe_month}", index=True)

    
    # # KS Tables
    # train_ks_table.to_excel(writer, sheet_name="Train_KS_Table", index=False)
    
    # # Train Results
    # train_evaluation_df.to_excel(writer, sheet_name='Train_Evaluation', index=False)
    # train_confusion_matrix_df.to_excel(writer, sheet_name='Train_Confusion_Matrix', index=True)

        
        
    

# Save the model

In [ ]:

import pickle

# Clean name1 and model
name1_clean = file_name_main.replace(" ", "_")
model_clean = models.replace(" ", "_")

  
# Save the Modle to file in the current working directory

Pkl_Filename = rf"{path_main}\{name1_clean}_{model_clean}_model_Pickle file.pkl"  

with open(Pkl_Filename, 'wb') as file:  
   pickle.dump(model, file)

#                               Model Comparision

# Folder Path and files

In [ ]:
threshold=optimal_threshold

# Target columns
target_columns=selected__target_column
# target_columns='TARGET'

# New Probability Column
new_probability="Probability"

# Existing model probability column
existing_probability="Pred_Score_1"



# All file location
folder_path_comparision=rf"D:\Pranto\Working\Collection Model\Collection New Model\3 to 3 Plus PV\Data\V1\3 to 3 Plus PV\Comparision"
# folder_path_comparision=rf"C:\Users\524264.CORP\Downloads"

# New model File name
# new_model_file_name=rf"2_TO_2_PLUS_ILCV_model_DEC23_TO_MAR25_Overall_Apply_binning_data_final_data_with_probability"


# new_model_file_name=rf"val_jan_ilcv"

# file1=rf"Final_Datav1_jan"

# files=[file1]
# month_name=['JAN25']

# Existing File name
file1=rf"Bkt_3_PV_JAN25_Score_File"

file2=rf"Bkt_3_PV_FEB25_Score_File"

file3=rf"Bkt_3_PV_MAR25_Score_File 1"



# Existing model files
files=[file1,file2,file3]
month_name=['JAN25','FEB25','MAR25']


# Export File name
export_file_name=rf"3_TO_3_PLUS_PV_model_JAN24_TO_MAR25_New Model"



## Read Files

In [ ]:
new_model_df=final_data

# try:
#     new_model_df=pd.read_csv(rf"{folder_path_comparision}\{new_model_file_name}.csv",encoding='latin-1')
# except:
#     new_model_df=pd.read_excel(rf"{folder_path_comparision}\{new_model_file_name}.xlsx")



print("######################################################################")

print("New model file Shape ",new_model_df.shape)
print("New model file Columns ",new_model_df.columns)

print("######################################################################")

dataframes={}

for file,month1 in zip(files,month_name):
    try:
        df_file=pd.read_csv(rf"{folder_path_comparision}\{file}.csv",encoding='latin-1')
        df_file['MONTH']=month1
        dataframes[month1]=df_file

        print("######################################################################")

        print(rf"{month1} file Shape ",df_file.shape)
        print(rf"{month1} Columns ",df_file.columns)

        print("######################################################################")

    except:
        df_file=pd.read_excel(rf"{folder_path_comparision}\{file}.xlsx")
        df_file['MONTH']=month1
        dataframes[month1]=df_file
        print("######################################################################")

        print(rf"{month1} file Shape ",df_file.shape)
        print(rf"{month1} Columns ",df_file.columns)

        print("######################################################################")


existing_model_df = pd.concat(dataframes.values(), ignore_index=True)

print("######################################################################")

print("Existing model shape",existing_model_df.shape)
print("Existing model columns",existing_model_df.columns)
print("######################################################################")


# Merge the new model data with the existing model data on CONNO and MONTH
merged_df = pd.merge(new_model_df, existing_model_df[['CONNO','MONTH',existing_probability]], on=["CONNO", "MONTH"], how="inner")


print("######################################################################")

print("After Merging model shape",merged_df.shape)
print("After Merging model columns",merged_df.columns)
print("######################################################################")

In [ ]:
target_columns

In [ ]:
merged_df

# Comparision

In [ ]:



# Evaluation function
def evaluate_model(y_true, y_prob, threshold=threshold, top_pct=0.3):
    df = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob})
    df['y_pred'] = (df['y_prob'] >= threshold).astype(int)
    cutoff = int(len(df) * top_pct)
    df_sorted = df.sort_values('y_prob', ascending=False).reset_index(drop=True)
    top_bads = df_sorted.iloc[:cutoff]['y_true'].sum()
    total_bads = df['y_true'].sum()
    capture_rate = top_bads / total_bads if total_bads else 0
    lift = (top_bads / cutoff) / (total_bads / len(df)) if total_bads else 0

    return {
        'AUC': roc_auc_score(y_true, y_prob),
        'Accuracy': accuracy_score(y_true, df['y_pred']),
        'Precision': precision_score(y_true, df['y_pred']),
        'Recall': recall_score(y_true, df['y_pred']),
        'F1 Score': f1_score(y_true, df['y_pred']),
        'Top_30%_Bad_Capture_Rate': capture_rate,
        'Lift_Top_30%': lift
    }


def calculate_group_metrics(df, target_column, probability_column):
    """
    Calculate metrics for High, Medium, and Low probability groups.
    """
    # Create probability-based groups
    df = df.copy()
    high_threshold = df[probability_column].quantile(0.7)
    low_threshold = df[probability_column].quantile(0.3)

    conditions = [
        df[probability_column] > high_threshold,
        (df[probability_column] > low_threshold) & (df[probability_column] <= high_threshold),
        df[probability_column] <= low_threshold
    ]
    choices = ['High', 'Medium', 'Low']
    df['Group'] = np.select(conditions, choices, default='Unknown')

    # Initialize metrics
    metrics = []

    total_population = len(df)
    total_bads = df[target_column] .sum()

    for group in ['High', 'Medium', 'Low']:
        group_data = df[df['Group'] == group]
        total = len(group_data)
        good = (group_data[target_column] == 0).sum()
        bad = (group_data[target_column] == 1).sum()
        bad_rate = bad / total if total > 0 else 0
        population_pct = total / total_population if total_population > 0 else 0
        bad_capture_rate = bad / total_bads if total_bads > 0 else 0
        lift = bad_rate / (total_bads / total_population) if total_population > 0 and total_bads > 0 else 0

        metrics.append({
            'Group': group,
            'Good': good,
            'Bad': bad,
            'Total': total,
            'Bad Rate': round(bad_rate, 4),
            'Bad Capture Rate': round(bad_capture_rate, 4),
            'Population %': round(population_pct, 4),
            'Lift': round(lift, 2)
        })

    return pd.DataFrame(metrics)




results = []
group_metrics_all_months = []

for month2 in month_name:
    temp = merged_df[merged_df['MONTH'] == month2]
    
    # For New Model metrics
    metrics_new = evaluate_model(temp[target_columns], temp[new_probability])
    metrics_new.update({'Month': month2, 'Model': 'NEW_MODEL'})
    results.append(metrics_new)
    
    # For Existing Model metrics
    metrics_existing = evaluate_model(temp[target_columns], temp[existing_probability])
    metrics_existing.update({'Month': month2, 'Model': 'EXISTING_MODEL'})
    results.append(metrics_existing)
    
    # Group Metrics for New Model
    group_metrics_new = calculate_group_metrics(temp, target_columns, new_probability)
    group_metrics_new['Model'] = 'NEW_MODEL'
    group_metrics_new['Month'] = month2
    group_metrics_all_months.append(group_metrics_new)

    # Group Metrics for Existing Model
    group_metrics_existing = calculate_group_metrics(temp, target_columns, existing_probability)
    group_metrics_existing['Model'] = 'EXISTING_MODEL'
    group_metrics_existing['Month'] = month2
    group_metrics_all_months.append(group_metrics_existing)

# Convert performance results into a DataFrame
results_df = pd.DataFrame(results)

results_df_V1=results_df[['Month', 'Model', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 
                  'Top_30%_Bad_Capture_Rate', 'Lift_Top_30%']]

# Combine the group metrics from all months
group_metrics_df = pd.concat(group_metrics_all_months, ignore_index=True)

group_metrics_df_V1=group_metrics_df[['Month', 'Model', 'Group', 'Good', 'Bad', 'Total', 'Bad Rate', 
                        'Bad Capture Rate', 'Population %', 'Lift']]





### Performance Metrics

In [ ]:

results_df_V1


#### Group Metrics

In [ ]:

group_metrics_df_V1

# Export Comparision Table

In [ ]:
   
# Create Excel with multiple sheets
excel_buffer = io.BytesIO()
with pd.ExcelWriter(rf"{folder_path_comparision}\{export_file_name}_{model_clean}_Model_Comparision.xlsx", engine='xlsxwriter') as writer:

    #PSI table
    group_metrics_df_V1.to_excel(writer, sheet_name='Group Metrics', index=False)
    # Comparision Table
    results_df_V1.to_excel(writer, sheet_name='Evaluation Metrics', index=False)


In [ ]:
# model_results=st.selectbox("",["Train Data","Test Data"],key='model_result')
#     if model_results == "Train Data":
#         # Streamlit UI: select train month
#         # Evaluation metrics
#         acc = accuracy_score(y_train, y_pred_train)
#         f1 = f1_score(y_train, y_pred_train)
#         recall = recall_score(y_train, y_pred_train)
#         bal_acc = balanced_accuracy_score(y_train, y_pred_train)
#         precision= precision_score(y_train, y_pred_train)

#         st.divider()

#         col1,col2,col3,col4,col5=st.columns(5)
#         with col1:
#             st.metric("Accuracy", f"{acc:.4f}")
#         with col2:
#             st.metric("Precision", f"{precision:.4f}")
#         with col3:
#             st.metric("F1 Score", f"{f1:.4f}")
#         with col4:
#             st.metric("Recall", f"{recall:.4f}")
#         with col5:
#             st.metric("Balanced Accuracy", f"{bal_acc:.4f}")
#         # # Display metrics
#         # st.write(f"**Accuracy:** `{acc:.4f}`")
#         # st.write(f"**F1 Score:** `{f1:.4f}`")
#         # st.write(f"**Recall:** `{recall:.4f}`")
#         # st.write(f"**Balanced Accuracy:** `{bal_acc:.4f}`")

#         # Create dataframe of metrics
#         metrics_df_train = pd.DataFrame({
#             'Metric': ['Accuracy', 'F1 Score', 'Recall', 'Balanced Accuracy'],
#             'Score': [acc, f1, recall, bal_acc]
#         })

#         # Show as table
#         # st.dataframe(metrics_df_train, use_container_width=True)

#         # # Export as CSV
#         # csv = metrics_df.to_csv(index=False).encode('utf-8')
#         # st.download_button("📥 Download Metrics CSV", csv, "train_metrics.csv", "text/csv")

#         st.markdown("---")
#         st.write("**Classification Report:**")
#         st.code(classification_report(y_train, y_pred_train))

#         # Confusion Matrix
#         st.write("**Confusion Matrix:**")
#         cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
#         disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train, display_labels=model.classes_)

#         fig, ax = plt.subplots()
#         disp_train.plot(ax=ax)
#         st.pyplot(fig)
#         pass

#     # For test months
#     else:
#         st.info("**`There is only train data for model building`**")
        


# st.divider()

# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# # View KS table
# st.markdown("<h2 style='color: #334fff; font-size: 30px;'>View KS table</h2>", unsafe_allow_html=True)

# # EXPANDER FOR KS TABLE
# with st.expander("**`View KS Table table`**",icon="📅"):

#     ks_table_train_test=st.selectbox("**`Select KS table for train and test data`**",["Train Data","Test Data"],key='ks_table')

#     if ks_table_train_test == "Train Data":
#         # Display KS table for training data
#         st.write("**KS Table for Training Data:**")
#         st.dataframe(ks_table_train, use_container_width=True)
#     else:
#         st.info("**`There is only train data for model building`**")


# # download all results
# ##############################################################################################

# ###
# # Train evaluation metrics
# train_evaluation_df = pd.DataFrame({
#     'Model Name': [model],
#     'Target_Variable': ['TARGET'],
#     'Optimal_Threshold': [optimal_threshold],
#     'Accuracy': [accuracy_score(y_train, y_pred_train)],
#     'Precision': [precision_score(y_train, y_pred_train)],
#     'Recall': [recall_score(y_train, y_pred_train)],
#     'F1_Score': [f1_score(y_train, y_pred_train)],
#     'Balanced_Accuracy': [balanced_accuracy_score(y_train, y_pred_train)],
#     'Variable Used in model':[top_features]
# })

# # ----------------------
# # 2. Create Train Confusion Matrix DataFrame
# # ----------------------

# cm_train = confusion_matrix(y_train, y_pred_train, labels=model.classes_)
# train_confusion_matrix_df = pd.DataFrame(
#     cm_train,
#     index=[f"Actual_{cls}" for cls in model.classes_],
#     columns=[f"Predicted_{cls}" for cls in model.classes_]
# )


# # ----------------------
# # 4. Prepare KS Tables (train and tests)
# # ----------------------

# # Train KS Table
# train_ks_table = ks_table_train



# # ----------------------
# # 5. Export everything into a ZIP
# # ----------------------

# # Clean name1 and model
# name1_clean = name1.replace(" ", "_")
# model_clean = models.replace(" ", "_")

# # Create in-memory zip file
# zip_buffer = io.BytesIO()
# # name1_clean = name1.replace(" ", "_")

# with zipfile.ZipFile(zip_buffer, "w") as zip_file:

#     # Final CSV
#     final_data_csv = final_data.to_csv(index=False)
#     zip_file.writestr(f"{name1_clean}_final_data_with_probability.csv", final_data_csv)

#     # Create Excel with multiple sheets
#     excel_buffer = io.BytesIO()
#     with pd.ExcelWriter(excel_buffer, engine='xlsxwriter') as writer:
        
#         # KS Tables
#         train_ks_table.to_excel(writer, sheet_name="Train_KS_Table", index=False)
        
#         # Train Results
#         train_evaluation_df.to_excel(writer, sheet_name='Train_Evaluation', index=False)
#         train_confusion_matrix_df.to_excel(writer, sheet_name='Train_Confusion_Matrix', index=True)

        
        
#     # Save Excel inside ZIP
#     zip_file.writestr(f"{name1_clean}_{model_clean}_model_summary.xlsx", excel_buffer.getvalue())

# # ----------------------
# # 6. Download Button in Streamlit
# # ----------------------

# st.success("✅ All data prepared successfully!")

# st.divider()

# st.download_button(
#     label="**`📩 Download Results ZIP ⬇`**",
#     data=zip_buffer.getvalue(),
#     file_name=f"{name1_clean}_model_results.zip",
#     mime="application/zip"
# )

# st.divider()

# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# #############%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [ ]:
# # Function to calculate metrics for probability percentiles (High, Medium, Low)
# def calculate_group_metrics(df, y_true, y_prob, percentile_1=0.3, percentile_2=0.7):
#     # df = pd.DataFrame({'y_true': y_true, 'y_prob': y_prob})
#     # Assign groups based on probability percentiles
#     low_threshold = df[y_prob].quantile(percentile_1)
#     high_threshold = df[y_prob].quantile(percentile_2)
    
#     df['Group'] = 'Low'
#     df.loc[df[y_prob] > high_threshold, 'Group'] = 'High'
#     df.loc[(df[y_prob] > low_threshold) & (df[y_prob] <= high_threshold), 'Group'] = 'Medium'
    
#     # Group-based metrics calculation
#     metrics = []
#     for group in ['High', 'Medium', 'Low']:
#         group_data = df[df['Group'] == group]
#         total = len(group_data)
#         good = (group_data['y_true'] == 0).sum()
#         bad = (group_data['y_true'] == 1).sum()
#         bad_rate = bad / total if total > 0 else 0
#         bad_capture_rate = bad / df['y_true'].sum() if df['y_true'].sum() > 0 else 0
#         population_pct = total / len(df)
#         lift = bad_capture_rate / (total / len(df)) if total > 0 else 0

#         metrics.append({
#             'Group': group,
#             'Good': good,
#             'Bad': bad,
#             'Total': total,
#             'Bad Rate': bad_rate,
#             'Bad Capture Rate': bad_capture_rate,
#             'Population%': population_pct,
#             'Lift': lift
#         })
    
#     return pd.DataFrame(metrics)

# Combine all metrics results
# results = []
# for month2 in month_name:
#     temp = merged_df[merged_df['MONTH'] == month2]
    
#     # For New Model metrics
#     metrics_new = evaluate_model(temp[target_columns], temp[new_probability])
#     metrics_new.update({'Month': month2, 'Model': 'NEW_MODEL'})
#     results.append(metrics_new)
    
#     # For Existing Model metrics
#     metrics_existing = evaluate_model(temp[target_columns], temp[existing_probability])
#     metrics_existing.update({'Month': month2, 'Model': 'EXISTING_MODEL'})
#     results.append(metrics_existing)

# # Convert results into a DataFrame
# results_df = pd.DataFrame(results)

# # Print the final table with all metrics for both models
# print("Model Performance Comparison (AUC, Accuracy, Precision, Recall, F1 Score, Top 30% Bad Capture Rate, Lift):")
# print(results_df[['Month', 'Model', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 
#                   'Top_30%_Bad_Capture_Rate', 'Lift_Top_30%']])

# # Now calculate and display metrics for High, Medium, and Low probability groups
# group_metrics_new = calculate_group_metrics(merged_df, target_columns, new_probability)
# group_metrics_new['Model'] = 'NEW_MODEL'

# group_metrics_existing = calculate_group_metrics(merged_df, target_columns, existing_probability)
# group_metrics_existing['Model'] = 'EXISTING_MODEL'

# # Combine the group metrics
# group_metrics_df = pd.concat([group_metrics_new, group_metrics_existing], ignore_index=True)

# # Display the group metrics table
# print("\nHigh, Medium, Low Group Metrics for Both Models:")
# group_metrics_df[['Model', 'Group', 'Good', 'Bad', 'Total', 'Bad Rate', 
#                         'Bad Capture Rate', 'Population %', 'Lift']]


In [ ]:
# # Comparison table
# # Sample data for development and validation (replace with actual data)
# development_data = pd.DataFrame({
#     'Decile':[1,2,3,4,5,6,7,8,9,10],
#     'Bad': ks_table_train['Good'],
#     'Good': ks_table_train['Bad']
# })

# validation_data = {
#     month: pd.DataFrame({
#         'Bad': ks_tables_test[f"KS_TABLE_{month}"]['Good'],
#         'Good': ks_tables_test[f"KS_TABLE_{month}"]['Bad']
#     })
#     for month in test_months
# }

# # validation_data

# # Helper function to calculate cumulative and derived values
# def calculate_metrics(data):
#     total_bad = data['Bad'].sum()
#     total_good = data['Good'].sum()

#     data['Total'] = data['Bad'] + data['Good']
#     data['Cumulative Bad'] = data['Bad'].cumsum()
#     data['Cumulative Good'] = data['Good'].cumsum()
#     data['Cumulative Bad%'] = data['Cumulative Bad'] / total_bad
#     data['Cumulative Good%'] = data['Cumulative Good'] / total_good
#     data['Good Rate'] = data['Good'] / data['Total']
#     data['%Bad'] = data['Bad'] / total_bad
#     data['%Good'] = data['Good'] / total_good
#     data['KS'] = data['Cumulative Good%'] - data['Cumulative Bad%']
#     # Add an 'AUC' column to the DataFrame for transparency
#     data['AUC'] =(data['Cumulative Good%'] / 2) * (data['Cumulative Bad%'])
#     for i in range(1, len(data)):
#         data.at[i, 'AUC'] = ((data['Cumulative Good%'][i] + data['Cumulative Good%'][i-1]) / 2) * (data['Cumulative Bad%'][i] - data['Cumulative Bad%'][i-1])
#     data['Rank'] = data['Good Rate'].rank(ascending=False)
#     data['Gini'] =''


#     return data, total_bad, total_good

# # Calculate for development data
# dev_data, dev_total_bad, dev_total_good = calculate_metrics(development_data)

# # Calculate for validation data (for each month)
# validation_metrics = {}
# for month, val_data in validation_data.items():
#     val_data, _, _ = calculate_metrics(val_data)
#     validation_metrics[month] = val_data



# # # Calculate ROB (Rank of Best) for each month
# for month, val_data in validation_metrics.items():
#     val_data['ROB'] = (dev_data['Rank'] != val_data['Rank']).astype(int)

# # # Calculate ROB (Rank of Best) for each month
# for month, val_data in validation_metrics.items():
#     val_data['Gini'] = ''


# # Combine Development Set and Validation Sets side by side
# combined_data = dev_data.copy()
# for month, val_data in validation_metrics.items():
#     # Align the validation data with the development data by index (assuming same number of rows)
#     val_data = val_data.rename(columns=lambda x: f"{x} ({month})")  # Rename columns to distinguish months
#     combined_data = pd.concat([combined_data, val_data], axis=1)


# # Add an overall total row at the end
# overall_totals = pd.DataFrame({
#         'Decile': ['Overall'],
#         'Bad': combined_data['Bad'].sum(axis=0),
#         'Good': combined_data['Good'].sum(axis=0),
#         'Total': combined_data['Total'].sum(axis=0),
#         'Good Rate': combined_data['Good'].sum(axis=0)/combined_data['Total'].sum(axis=0),
#         '%Bad': combined_data['%Bad'].sum(axis=0),
#         '%Good': combined_data['%Good'].sum(axis=0),
#         'KS': combined_data['KS'].max(),
#         'AUC': combined_data['AUC'].sum(axis=0),
#         'Gini': abs(1 - 2 * combined_data['AUC'].sum(axis=0)),

#         **{f"Bad ({month})": combined_data[f"Bad ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"Good ({month})": combined_data[f"Good ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"Total ({month})": combined_data[f"Total ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"Good Rate ({month})": combined_data[f"Good ({month})"].sum(axis=0)/combined_data[f"Total ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"%Bad ({month})": combined_data[f"%Bad ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"%Good ({month})": combined_data[f"%Good ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"KS ({month})": combined_data[f"KS ({month})"].max() for month in validation_metrics},
#         **{f"AUC ({month})": combined_data[f"AUC ({month})"].sum(axis=0) for month in validation_metrics},
#         **{f"ROB ({month})": combined_data[f"ROB ({month})"].sum(axis=0)/combined_data[f"ROB ({month})"].count() for month in validation_metrics},
#         **{f"Gini ({month})": (abs(1-2*(combined_data[f"AUC ({month})"].sum(axis=0)))) for month in validation_metrics}
# })

# # Append the overall totals row at the end of the combined data
# combined_data_with_totals = pd.concat([combined_data, overall_totals], ignore_index=True)

# # Display the final combined data with the overall totals row
# combined_data_with_totals
